In [1]:
from IPython import display

import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dset

import numpy as np

import time
from visdom import Visdom

from lib.VisdomWrapper import *
from lib.DataManager import *

In [2]:
torch.manual_seed(2)
batch_size = 512
num_epochs = 30

loss = nn.CrossEntropyLoss()
vis = VisdomController()

mnist = dset.MNIST('input', train=True, download=True, transform=T.ToTensor())
balanced_train = DataLoader(mnist, batch_size =batch_size, shuffle=True)
mnist_low_zero = get_unbalanced_mnist([.01, .1, .1, .1, .1, .1, .1, .1, .1, .1], batch_size=batch_size)
mnist_high_nine = get_unbalanced_mnist([.1, .1, .1, .1, .1, .1, .1, .1, .1, 1], batch_size=batch_size)

mnist_test=dset.MNIST('input', train=False, download=True, transform=T.ToTensor())
balanced_test = DataLoader(mnist_test)

Setting up a new session...


In [3]:
def plot_frequency(data_loader, title):
    x = np.arange(10)
    cnts = torch.zeros(10)
    for _, y in data_loader:
        cnts += torch.bincount(y)
    vis.CreateStaticBarPlot(cnts, x, title, "Class", "Count", "FreqPlot")

In [4]:
def build_classifier():
    return nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=5, stride=1),
        nn.LeakyReLU(0.01),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 64, kernel_size=5, stride=1),
        nn.LeakyReLU(0.01),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(4*4*64, 4*4*64, bias=True),
        nn.LeakyReLU(0.01),
        nn.Linear(4*4*64, 10, bias=True)
    )

In [5]:
def get_optimizer(model):
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    return optimizer

In [6]:
def train_classifier(classifier, optimizer, data, key="Loss"):
    for epoch in range(num_epochs):
        for n_batch, (x, y) in enumerate(data):
            if len(x) != batch_size:
                continue
            optimizer.zero_grad
            x = x.cuda()
            y = y.cuda()
            scores = classifier(x)
            out = loss(scores, y)
            out.backward()
            optimizer.step()
        display.clear_output(True)
        print("Epoch {}, {} / {}".format(epoch, n_batch, len(data)))
        print("Loss: ", out.item())
        vis.loss_axis = epoch
        vis.PlotLoss(key, out.item())
            

In [7]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()


    print('Accuracy: {}/{} ({:.0f}%)\n'.format(correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
balanced_net = build_classifier().cuda()
balanced_net.train()
train_classifier(balanced_net, get_optimizer(balanced_net), balanced_train, key="Balanced")


Epoch 29, 117 / 118
Loss:  0.09371086955070496


In [16]:
low_zero_net = build_classifier().cuda()
low_zero_net.train()
train_classifier(low_zero_net, get_optimizer(low_zero_net), mnist_low_zero, key="Low Zero")

Epoch 29, 117 / 118
Loss:  0.06981930136680603


In [13]:
high_nine_net = build_classifier().cuda()
high_nine_net.train()
train_classifier(high_nine_net, get_optimizer(high_nine_net), mnist_high_nine, key="High Nine")

Epoch 29, 117 / 118
Loss:  0.038997747004032135


In [9]:
device = torch.device("cuda")
balanced_net.eval()
test(balanced_net, device, balanced_test)

Accuracy: 9755/10000 (98%)



In [17]:
device = torch.device("cuda")
low_zero_net.eval()
test(low_zero_net, device, balanced_test)

Accuracy: 8964/10000 (90%)



In [14]:
device = torch.device("cuda")
high_nine_net.eval()
test(high_nine_net, device, balanced_test)

Accuracy: 8822/10000 (88%)



In [26]:
# plot_frequency(mnist_low_zero, "Low Zero Class Counts")
plot_frequency(mnist_high_nine, "High Nine Class Counts")

In [23]:
nb_classes = 10
model=low_zero_net

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(balanced_test):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
confusion_matrix = confusion_matrix.numpy()
np.fill_diagonal(confusion_matrix, 0)
vis.PlotHeatMap(confusion_matrix, "Low Zero Confusion", False)
#y is correct, x is predicted

In [21]:
print(confusion_matrix[5][9])

119.0
